In [1]:
import numpy as np
import pandas as pd
from os.path import join
from skimage.measure import regionprops_table
import matplotlib.pyplot as plt
import os
from os import listdir
from os.path import isfile, join

In [2]:
#nuclei and cellSizePath
nucleiPath = "/Volumes/Aortas/Aorta/segmentations/cellSize/tiles/tilesNuclei"
cellSizePath = "/Volumes/Aortas/Aorta/segmentations/cellSize/tiles/tilesCellSize"

In [3]:
nucFolders = [x[0] for x in os.walk(nucleiPath)]
nucFolders = sorted(nucFolders)

In [4]:
def nucleiCheck(cytoPath, nucleiPath, borderpixel, accuracy):
    #input: path cytoplasm, path nuclei; pixel tube for border pixel
    nucleiSegmentation = np.load(nucleiPath, allow_pickle=True).item()
    cytoSegmentation = np.load(cytoPath, allow_pickle=True).item()
    nucleiMasks = nucleiSegmentation['masks']
    cytoMasks = cytoSegmentation['masks']
    cytoOutlines = cytoSegmentation['outlines']
    imgCyto = cytoSegmentation['img']
    excludeX = np.arange(imgCyto.shape[1] - borderpixel + 1 ,imgCyto.shape[1] +1)
    downValues = np.arange(0,borderpixel)
    excludeX = np.concatenate((downValues, excludeX))
    excludeY = np.arange(imgCyto.shape[0] - borderpixel +1 ,imgCyto.shape[0] +1)
    excludeY = np.concatenate((downValues, excludeY))
    # nucleicheck
    uniqueCells = np.unique(cytoMasks)
    for c in uniqueCells:
        if c != 0:
            cellCoord = np.where(cytoMasks==c)
            checkY = np.isin(cellCoord[0],excludeY)
            checkX = np.isin(cellCoord[1],excludeX)
            if np.any(checkY):
                cytoMasks[cellCoord[0], cellCoord[1]] = 0
                cytoOutlines[cellCoord[0], cellCoord[1]] = 0
            elif np.any(checkX):
                cytoMasks[cellCoord[0], cellCoord[1]] = 0
                cytoOutlines[cellCoord[0], cellCoord[1]] = 0
            else:
                nuclei = nucleiMasks[cellCoord[0], cellCoord[1]]
                uniqueNuclei = np.unique(nuclei)
                if len(np.unique(nuclei)) <=1:
                    cytoMasks[cellCoord[0], cellCoord[1]] = 0
                    cytoOutlines[cellCoord[0], cellCoord[1]] = 0
                elif len(np.unique(nuclei)) > 3:
                    cytoMasks[cellCoord[0], cellCoord[1]] = 0
                    cytoOutlines[cellCoord[0], cellCoord[1]] = 0
                else:
                    coverage = 0.0
                    for n in uniqueNuclei:
                        if n!= 0:
                            nucleiCoord = np.where(nucleiMasks==n)
                            #How many pixels does this nucleus have:
                            cytoNumber = cytoMasks[nucleiCoord[0], nucleiCoord[1]]
                            #occurence of nucleus pixel in cyto
                            cytoNumberNuclei = np.bincount(cytoNumber)[c]
                            # max percentage of how many pixels of nuc are in cellSize
                            if (cytoNumberNuclei/len(cytoNumber) > coverage):
                                coverage = cytoNumberNuclei/len(cytoNumber)

                    if (coverage < accuracy):
                        cytoMasks[cellCoord[0], cellCoord[1]] = 0
                        cytoOutlines[cellCoord[0], cellCoord[1]] = 0
    return cytoOutlines, cytoMasks, imgCyto

In [ ]:
dfResults = pd.DataFrame()
dicNoValue = {"age":[],"aorta":[],"file":[]}
for nucFolder in nucFolders:
    os.chdir(nucFolder)
    files = [f for f in listdir(nucFolder) if isfile(join(nucFolder, f)) if ".DS_Store" not in f if ".npy" in f if "._" not in f]
    for file in files:
        aorta = nucFolder.split("/")[-1]
        age = nucFolder.split("/")[-2]
        if os.path.exists(join(cellSizePath, age, aorta, "15Diam", file)):
            cellSizeStartPath = join(cellSizePath, age, aorta, "15Diam", file)
            cellSizePaths = [join(cellSizePath, age, aorta, "20Diam", file), join(cellSizePath, age, aorta, "25Diam", file), join(cellSizePath, age, aorta, "30Diam", file)]
            cytoOutlines, cytoMasks, imgCyto = nucleiCheck(cellSizeStartPath, join(nucFolder, file), 5, 0.75)
            maxID = np.max(np.unique(cytoOutlines))
            for cellSizePathAna in cellSizePaths:
                cytoOutlinesNew, cytoMasksNew, imgCytoNew = nucleiCheck(cellSizePathAna, join(nucFolder, file), 5, 0.75)
                for c in np.unique(cytoOutlinesNew):
                    if c != 0:
                        cellCoord = np.where(cytoMasksNew==c)
                        segmentationOld = cytoMasks[cellCoord]
                        if len(np.unique(segmentationOld) == 1) & (np.unique(segmentationOld)[0] == 0):
                            maxID += 1
                            cytoMasks[cellCoord] = maxID
                            cytoOutlines[np.where(cytoOutlinesNew == c)] = maxID
                        else:
                            for cellOld in np.unique(segmentationOld):
                                if cellOld != 0:
                                    outlineMeanNew = np.mean(imgCyto[np.where(cytoOutlinesNew == c)])
                                    if len(imgCyto[np.where(cytoOutlines == cellOld)]) > 0:
                                        outlineMeanOld = np.mean(imgCyto[np.where(cytoOutlines == cellOld)])
                                    else:
                                        outlineMeanOld = 0.0
                                    if outlineMeanNew > outlineMeanOld:
                                        maxID += 1
                                        cytoMasks[np.where(cytoOutlines == cellOld)] = 0
                                        cytoMasks[cellCoord] = maxID
                                        cytoOutlines[np.where(cytoOutlinesNew == c)] = maxID
            prop_dict = regionprops_table(cytoMasks, properties=['label','area', 'centroid', 'major_axis_length', 'minor_axis_length', "orientation"])
            dfHelp = pd.DataFrame(prop_dict)
            dfHelp['age'] = age
            dfHelp['aorta'] = aorta
            dfHelp['file'] = file
            dfResults = dfResults.append(dfHelp)
        else:
            dicNoValue["age"].append(age)
            dicNoValue["aorta"].append(aorta)
            dicNoValue["file"].append(file)

/var/folders/6r/4p5g0w_j4jv11sr24nv1b0140000gn/T/ipykernel_2243/1380316879.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfResults = dfResults.append(dfHelp)
/var/folders/6r/4p5g0w_j4jv11sr24nv1b0140000gn/T/ipykernel_2243/1380316879.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfResults = dfResults.append(dfHelp)
/var/folders/6r/4p5g0w_j4jv11sr24nv1b0140000gn/T/ipykernel_2243/1380316879.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfResults = dfResults.append(dfHelp)
/var/folders/6r/4p5g0w_j4jv11sr24nv1b0140000gn/T/ipykernel_2243/1380316879.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfResults = dfResults

In [7]:
dfResults.to_csv("/Volumes/Aortas/Aorta/results/cellShapesNew.csv")